In [1]:
import numpy as np
import pandas as pd
import sys
import os 

from os import path
from eppy import eppy
from eppy import hvacbuilder
from eppy import modeleditor
from eppy.modeleditor import IDF

In [2]:
# this function determines what idf version and connects the idd for eppy
def eppy_connection(idffile):
    fp = open(idffile)
    lines=fp.readlines()
    vers = ""
    vers = vers.join(lines[2:12])
    vers
    if vers.find('8.7') != -1:
        print('Using EnergyPlus version 8.7')
        iddfile = 'C:\\EnergyPlusV8-7-0\\Energy+.idd'
        IDF.setiddname(iddfile)
        idf = IDF(idffile)
        return idf
    elif vers.find('8.8') != -1:
        print('Using EnergyPlus version 8.8')
        iddfile = 'C:\\EnergyPlusV8-8-0\\Energy+.idd'
        IDF.setiddname(iddfile)
        idf = IDF(idffile)
        return idf
    elif vers.find('8.9') != -1:
        print('Using EnergyPlus version 8.9')
        iddfile = 'C:\\EnergyPlusV8-9-0\\Energy+.idd'
        IDF.setiddname(iddfile)
        idf = IDF(idffile)
        return idf
    elif vers.find('9.0') != -1:
        print('Using EnergyPlus version 9.0.1')
        iddfile = 'C:\\EnergyPlusV9-0-1\\Energy+.idd'
        IDF.setiddname(iddfile)
        idf = IDF(idffile)
        return idf
    else:
        print('IDF Type not found on line 3')

In [3]:
#define paths
commonSlab_path = os.path.join(path.abspath(path.join('',"../..")), "DOE_LRMF\\SEED_Models\\common_hp_slab_IECC_2012_V8.7.idf")
commonBsmt_path = os.path.join(path.abspath(path.join('',"../..")), "DOE_LRMF\\SEED_Models\\common_hp_heatedbsmtIECC_2012_V8.7.idf")
gardenSlab_path = os.path.join(path.abspath(path.join('',"../..")), "DOE_LRMF\\SEED_Models\\garden_hp_slab_IECC_2012_V8.7.idf")
gardenBsmt_path = os.path.join(path.abspath(path.join('',"../..")), "DOE_LRMF\\SEED_Models\\garden_hp_heatedbsmtIECC_2012_V8.7.idf")

#make connections
commonSlab = eppy_connection(commonSlab_path)
commonBsmt = eppy_connection(commonBsmt_path)
gardenSlab = eppy_connection(gardenSlab_path)
gardenBsmt = eppy_connection(gardenBsmt_path)

# create groups
idfs = [commonSlab, commonBsmt, gardenSlab, gardenBsmt]
common = [commonSlab, commonBsmt]
garden = [gardenSlab, gardenBsmt]
bsmt = [commonBsmt, gardenBsmt]
slab = [commonSlab, gardenSlab]

Using EnergyPlus version 8.7
Using EnergyPlus version 8.7
Using EnergyPlus version 8.7
Using EnergyPlus version 8.7


In [4]:
## LIST SURFACES TO MODIFY

commonFloorSurfaces = ['Floor_unit1_FrontRow_BottomFloor', 'Floor_unit2_FrontRow_BottomFloor', 
            'Floor_unit3_FrontRow_BottomFloor', 'Floor_unit1_BackRow_BottomFloor', 
            'Floor_unit2_BackRow_BottomFloor', 'Floor_unit3_BackRow_BottomFloor', 
            'g Floor C']

gardenFloorSurfaces = ['Floor_unit1_FrontRow_BottomFloor', 'Floor_unit2_FrontRow_BottomFloor', 
            'Floor_unit3_FrontRow_BottomFloor', 'Floor_unit1_BackRow_BottomFloor', 
            'Floor_unit2_BackRow_BottomFloor', 'Floor_unit3_BackRow_BottomFloor']

commonSlabSurfaces = ['BGFloor']

gardenSlabSurfaces = ['BGFloor']

In [5]:
## CREATE NEW IDF OBJECTS

#common
for i in range(0,len(idfs)):
    idfs[i].newidfobject('Material')
    idfs[i].idfobjects['Material'][-1].Name = 'slab_consol_layer'
    idfs[i].idfobjects['Material'][-1].Roughness = 'Rough'
    idfs[i].idfobjects['Material'][-1].Thickness = 0.23495
    idfs[i].idfobjects['Material'][-1].Conductivity = 0.0485233
    idfs[i].idfobjects['Material'][-1].Density = 55.074
    idfs[i].idfobjects['Material'][-1].Specific_Heat = 916.9311


    idfs[i].newidfobject('Construction')
    idfs[i].idfobjects['Construction'][-1].Name = 'Slab'
    idfs[i].idfobjects['Construction'][-1].Outside_Layer = 'slab_consol_layer'
    idfs[i].idfobjects['Construction'][-1].Layer_2 = 'Plywood_3/4in'
    idfs[i].idfobjects['Construction'][-1].Layer_3 = 'Carpet_n_pad'

In [7]:
## LOOP THROUGH SURFACES AND CHANGE CONSTRUCTIONS

# common SOG Slab
surfaces = commonSlab.idfobjects['BuildingSurface:Detailed']
for i in range(0, len(commonFloorSurfaces)):
    
    # grab surface from list of surfaces
    surface = [x for x in surfaces if x.Name == commonFloorSurfaces[i]][0]
    surface.Construction_Name = 'Slab'

    #display changes
    print('Common SOG: Changed '+surface.Name+' construction to '+surface.Construction_Name)
    
# garden SOG Slab
surfaces = gardenSlab.idfobjects['BuildingSurface:Detailed']
for i in range(0, len(gardenFloorSurfaces)):
    
    # grab surfaces from list of surfaces
    surface = [x for x in surfaces if x.Name == gardenFloorSurfaces[i]][0]
    surface.Construction_Name = 'Slab'
    
    #display changes
    print('Garden SOG: Changed '+surface.Name+' construction to '+surface.Construction_Name)
    
# common basement Slab
surfaces = commonBsmt.idfobjects['BuildingSurface:Detailed']
for i in range(0, len(commonSlabSurfaces)):
    
    # grab surface from list of surfaces
    surface = [x for x in surfaces if x.Name == commonSlabSurfaces[i]][0]
    surface.Construction_Name = 'Slab'

    #display changes
    print('Common bsmt: Changed '+surface.Name+' construction to '+surface.Construction_Name)
    
# garden basement Slab
surfaces = gardenBsmt.idfobjects['BuildingSurface:Detailed']
for i in range(0, len(gardenSlabSurfaces)):
    
    # grab surfaces from list of surfaces
    surface = [x for x in surfaces if x.Name == gardenSlabSurfaces[i]][0]
    surface.Construction_Name = 'Slab'
    
    #display changes
    print('Garden bsmt: Changed '+surface.Name+' construction to '+surface.Construction_Name) 

Common SOG: Changed Floor_unit1_FrontRow_BottomFloor construction to Slab
Common SOG: Changed Floor_unit2_FrontRow_BottomFloor construction to Slab
Common SOG: Changed Floor_unit3_FrontRow_BottomFloor construction to Slab
Common SOG: Changed Floor_unit1_BackRow_BottomFloor construction to Slab
Common SOG: Changed Floor_unit2_BackRow_BottomFloor construction to Slab
Common SOG: Changed Floor_unit3_BackRow_BottomFloor construction to Slab
Common SOG: Changed g Floor C construction to Slab
Garden SOG: Changed Floor_unit1_FrontRow_BottomFloor construction to Slab
Garden SOG: Changed Floor_unit2_FrontRow_BottomFloor construction to Slab
Garden SOG: Changed Floor_unit3_FrontRow_BottomFloor construction to Slab
Garden SOG: Changed Floor_unit1_BackRow_BottomFloor construction to Slab
Garden SOG: Changed Floor_unit2_BackRow_BottomFloor construction to Slab
Garden SOG: Changed Floor_unit3_BackRow_BottomFloor construction to Slab
Common bsmt: Changed BGFloor construction to Slab
Garden bsmt: Chan

In [8]:
## save idfs
[x.save() for x in idfs]

[None, None, None, None]